<a href="https://colab.research.google.com/github/Shyam-Khokhariya/ML-Python/blob/master/C15_CNN_Mnist_Tf_Layer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import tensorflow as tf
import numpy as np
import os

In [0]:
mnist=np.load("mnist_scaled.npz")
X_train,y_train,X_test,y_test=[mnist[f] for f in mnist.files]

In [4]:
X_valid,y_valid=X_train[50000:,:],y_train[50000:]
X_train,y_train=X_train[:50000,:],y_train[:50000]
print(X_train.shape,y_train.shape)
print(X_valid.shape,y_valid.shape)
print(X_test.shape,y_test.shape)

(50000, 784) (50000,)
(10000, 784) (10000,)
(10000, 784) (10000,)


In [0]:
def batch_generator(X,y,batch_size=64,shuffle=False,random_seed=None):
  idx=np.arange(y.shape[0])
  if shuffle:
    rng=np.random.RandomState(random_seed)
    rng.shuffle(idx)
    X=X[idx]
    y=y[idx]
  for i in range(0,X.shape[0],batch_size):
    yield (X[i:i+batch_size,:],y[i:i+batch_size])


In [0]:
mean_val=np.mean(X_train,axis=0)
std_val=np.std(X_train)

X_train_centered=(X_train-mean_val)/std_val
X_valid_centered=(X_valid-mean_val)/std_val
X_test_centered=(X_test-mean_val)/std_val


In [0]:
class ConvNN(object):
    def __init__(self,batchsize=64,epochs=20,
                 learning_rate=1e-4,dropout_rate=0.5,
                 shuffle=True,random_seed=None):
        np.random.seed(random_seed)
        self.batchsize=batchsize
        self.epochs=epochs
        self.learning_rate=learning_rate
        self.dropout_rate=dropout_rate
        self.shuffle=shuffle

        g=tf.Graph()
        with g.as_default():
            #Set random Seed
            tf.set_random_seed(random_seed)

            #Building CNN
            self.build()

            #Initializer
            self.init_op=tf.global_variables_initializer()

            #Saver
            self.saver=tf.train.Saver()

        #Creating Session
        self.sess=tf.Session(graph=g)

    def build(self):
        #Placeholder For X & y
        tf_x=tf.placeholder(tf.float32,shape=[None,784],name="tf_x")
        tf_y=tf.placeholder(tf.int32,shape=[None],name="tf_y")
        is_train=tf.placeholder(tf.bool,shape=(),name="is_train")

        #reshaping x to 4D tensor
        tf_x_image=tf.reshape(tf_x,shape=[-1,28,28,1],name="input_x_2dimages")

        #One-Hot Encoding
        tf_y_onehot=tf.one_hot(indices=tf_y,depth=10,dtype=tf.float32,name="input_y_onehot")

        #1st Layer Conv_1
        h1=tf.layers.conv2d(tf_x_image,filters=32,kernel_size=(5,5),activation=tf.nn.relu)

        ##Max Pooling
        h1_pool=tf.layers.max_pooling2d(h1,pool_size=(2,2),strides=(2,2))

        #2nd Layer Conv_2
        h2=tf.layers.conv2d(h1_pool,filters=64,kernel_size=(5,5),activation=tf.nn.relu)

        #Max Pooling
        h2_pool=tf.layers.max_pooling2d(h2,pool_size=(2,2),strides=(2,2))

        #3rd Layer Fully Connected
        input_shape=h2_pool.get_shape().as_list()
        n_input_units=np.prod(input_shape[1:])
        h2_pool_flat=tf.reshape(h2_pool,shape=[-1,n_input_units])
        h3=tf.layers.dense(h2_pool_flat,1024,activation=tf.nn.relu)

        #Dropout
        h3_drop=tf.layers.dropout(h3,rate=self.dropout_rate,training=is_train)

        #4th Layer Fully Connected (Linear Activation)
        h4=tf.layers.dense(h3_drop,10,activation=None)

        ##Prediction
        prediction={"probabilities":tf.nn.softmax(h4,name="probabilities"),
                    "labels":tf.cast(tf.argmax(h4,axis=1),tf.int32,name="labels")}

        #Loss function & Optimization
        cross_entropy_loss=tf.reduce_mean(
            tf.nn.softmax_cross_entropy_with_logits(logits=h4,labels=tf_y_onehot),
            name="cross_entropy_loss")

        #Optimizer
        optimizer=tf.train.AdamOptimizer(self.learning_rate)
        optimizer=optimizer.minimize(cross_entropy_loss,name="train_op")

        #Finding Accuracy
        correct_predictions=tf.equal(prediction['labels'],tf_y,name="correct_pred")
        accuracy=tf.reduce_mean(tf.cast(correct_predictions,tf.float32),name="accuracy")

    def save(self,epoch,path="./tflayers-model/"):
        if not os.path.isdir(path):
            os.makedirs(path)
        print("Saving model at %s"%path)
        self.saver.save(self.sess,os.path.join(path,"model.ckpt"),global_step=epoch)

    def load(self,epoch,path):
        print("Loading from %s"%path)
        self.saver.restore(self.sess,os.path.join(path,"model.ckpt-%d")%epoch)

    def train(self,training_set,validation_set=None,initialize=True):
        #initialize variables
        if initialize:
            self.sess.run(self.init_op)
        self.train_cost=[]
        X_data=np.array(training_set[0])
        y_data=np.array(training_set[1])

        for epoch in range(1,self.epochs+1):
            batch_gen=batch_generator(X_data,y_data,shuffle=self.shuffle)

            avg_loss=0.0
            for i,(batch_x,batch_y) in enumerate(batch_gen):
                feed={"tf_x:0":batch_x,
                      "tf_y:0":batch_y,
                      "is_train:0":True} #for Dropout
                loss,_=self.sess.run(["cross_entropy_loss:0","train_op"],feed_dict=feed)
                avg_loss+=loss

            print("Epoch %02d: Training Average Loss : %7.3f"%(epoch,avg_loss),end=" ")

            if validation_set is not None:
                feed={"tf_x:0":validation_set[0],
                      "tf_y:0":validation_set[1],
                      "is_train:0":False}
                valid_acc=self.sess.run("accuracy:0",feed_dict=feed)
                print("Validation Accuracy : %7.3f"%valid_acc)
            else:
                print()

    def predict(self,X_test,return_prob=False):
        feed={"tf_x:0":X_test,
              "is_train:0":False}
        if return_prob:
            return self.sess.run("probabilities:0",feed_dict=feed)
        else:
            return self.sess.run("labels:0",feed_dict=feed)


In [10]:
cnn=ConvNN(random_seed=123)
cnn.train(training_set=(X_train_centered,y_train),validation_set=(X_valid_centered,y_valid),
          initialize=True)
cnn.save(epoch=20)


Epoch 01: Training Average Loss : 261.666 Validation Accuracy :   0.975
Epoch 02: Training Average Loss :  72.429 Validation Accuracy :   0.983
Epoch 03: Training Average Loss :  49.559 Validation Accuracy :   0.986
Epoch 04: Training Average Loss :  38.993 Validation Accuracy :   0.988
Epoch 05: Training Average Loss :  31.129 Validation Accuracy :   0.989
Epoch 06: Training Average Loss :  26.671 Validation Accuracy :   0.989
Epoch 07: Training Average Loss :  22.400 Validation Accuracy :   0.990
Epoch 08: Training Average Loss :  19.730 Validation Accuracy :   0.991
Epoch 09: Training Average Loss :  17.438 Validation Accuracy :   0.991
Epoch 10: Training Average Loss :  15.663 Validation Accuracy :   0.991
Epoch 11: Training Average Loss :  12.779 Validation Accuracy :   0.991
Epoch 12: Training Average Loss :  11.211 Validation Accuracy :   0.992
Epoch 13: Training Average Loss :   9.554 Validation Accuracy :   0.992
Epoch 14: Training Average Loss :   8.913 Validation Accuracy : 

In [11]:
cnn2=ConvNN(random_seed=123)
cnn2.load(epoch=20,path="./tflayers-model/")
print(cnn2.predict(X_test_centered[:10,:]))


Loading from ./tflayers-model/
Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from ./tflayers-model/model.ckpt-20
[7 2 1 0 4 1 4 9 5 9]


In [12]:
pred=cnn2.predict(X_test_centered)
print("Test Accuracy : %.2f"%(100 * np.sum(y_test==pred)/len(y_test)))


Test Accuracy : 99.29
